<a href="https://colab.research.google.com/github/sarahpursley/dsci551-assignment2/blob/main/GPT_2_Model_FULL_EMAILS_AND_REPLIES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

**1. Set up versions and packages as well as start the GPU** 

**GPU:** Use either a Nvidia T4 GPU or an Nvidia K80 GPU and verify which GPU.


In [ ]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

!nvidia-smi

Fri Apr  9 01:21:59 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P0    30W /  70W |   8484MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

**2. download_gpt2 small model (124M)**



In [ ]:
gpt2.download_gpt2(model_name="124M")

Fetching checkpoint: 1.05Mit [00:00, 219Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 6.12Mit/s]
Fetching hparams.json: 1.05Mit [00:00, 194Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:51, 9.67Mit/s]
Fetching model.ckpt.index: 1.05Mit [00:00, 181Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 7.86Mit/s]
Fetching vocab.bpe: 1.05Mit [00:00, 9.49Mit/s]


**File upload from local machine: only upload once into files**
Note: cannot pull from github because our final json from assignment one was too large. There is a zip file under datasets/fraudulent_emails_v2 to download a local copy and then upload here.

In [ ]:
import os
from google.colab import files

os.chdir('/content')
uploaded = files.upload()

Saving fraudulent_emails_v2.json to fraudulent_emails_v2.json


In [ ]:
# break out the different attack types
# we already had a singular attack type assigned in our assignment 1 file 
# using 'se_tag' as the key 
import json
import re
messages = json.load(open('fraudulent_emails_v2.json',))

recon_train = []
se_train = []
phish_train = []
mal_train = []

whitespace_ptrn = re.compile(r'\s')

# append corresponding se_tag to one of the attack types (lists initialized above)
for message in messages:
    if 'se_tag' in message:
        # we also want the ttr generated text and cleaned up for later
        ttr_dict = message['TTR_text']
        t = ttr_dict['ttr_text']
        norm_content = whitespace_ptrn.sub(' ', t).encode('utf-8')

        if t != '':
            if message['se_tag'] == 'recon':
                t = t.lstrip().strip()
                recon_train.append(norm_content.decode("utf-8"))

            if message['se_tag'] == 'social_engineering':
                se_train.append(norm_content.decode("utf-8"))

            if message['se_tag'] == 'phishing':
                phish_train.append(norm_content.decode("utf-8"))

            elif message['se_tag'] == 'malware':
                mal_train.append(norm_content.decode("utf-8"))

            else:
                continue
    else:
        print('hi')

# load local json file of reply examples used for part 10
# will include in final report
reply_emails = json.load(open('replyEmails.json',))
    

print("Recon train:", len(recon_train))
print("SE messages:", len(se_train))
print("Phishing messages: ", len(phish_train))
print("Malware messages: ", len(mal_train))
print("Reply messages: ", len(reply_emails))

In [ ]:
# download each attack type list of messages to a text file
with open("r_train_.txt", "w") as output:
    for listitem in recon_train:
        output.write('%s\n' % listitem)             

with open("s_train.txt", "w") as output:
    for listitem in se_train:
        output.write('%s\n' % listitem)             

with open("p_train.txt", "w") as output:
    for listitem in phish_train:
        output.write('%s\n' % listitem)             

with open("m_train.txt", "w") as output:
    for listitem in mal_train:
        output.write('%s\n' % listitem)             

with open("reply_train.txt", "w") as output:
    output.write(str(reply_emails))

# Finetune GPT2 Model

## Recon Model


In [ ]:
### IF YOU NEED TO RESTART THE RUNTIME WHICH YOU DO AFTER EACH GENERATE/FINETUNE SESSION
# local files are saved
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

!nvidia-smi

TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Fri Apr  9 01:18:12 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4    

In [ ]:
# write file name and define parameters
# used max steps
# refreshed the 124M model each run as to keep the attack models different
file_name = "r_train_.txt"
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='124M',
              steps=10,
              restore_from='fresh',
              run_name='recon',
              print_every=10,
              sample_every=100,
              save_every=200
              )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Loading checkpoint models/124M/model.ckpt
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:01<00:00,  1.59s/it]


dataset has 284620 tokens
Training...
[10 | 28.67] loss=3.08 avg=3.08
Saving checkpoint/recon/model-10


In [ ]:
# write file name and define parameters
# used max steps
# refreshed the 124M model each run as to keep the attack models different
file_name = "recon_train.txt"
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='124M',
              steps=1000,
              restore_from='fresh',
              run_name='recon',
              print_every=10,
              sample_every=100,
              save_every=200
              )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Loading checkpoint models/124M/model.ckpt
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:01<00:00,  1.56s/it]


dataset has 284886 tokens
Training...
[10 | 28.46] loss=3.40 avg=3.40
[20 | 50.25] loss=2.66 avg=3.03
[30 | 72.70] loss=2.81 avg=2.96
[40 | 96.07] loss=2.70 avg=2.89
[50 | 120.56] loss=2.87 avg=2.89
[60 | 144.53] loss=2.50 avg=2.82
[70 | 167.98] loss=2.44 avg=2.76
[80 | 191.71] loss=1.73 avg=2.63
[90 | 215.60] loss=1.96 avg=2.55
[100 | 239.31] loss=1.77 avg=2.47
======== SAMPLE 1 ========
 assist to furnish the needed particulars, and the Attorney-at-law will present the relevant documents and affidavits of the said parties to back up his proposition.All parties will be provided with the following documents/firma- notes as proof of the said amount: (1) In-kind gratuities of US$100.00 (2) In-kind gratuities of US$5,000,000 (3) In-kind gratuities of US$500,000.00 (4) In-kind gratuities of US$50,000.00 (5) In-kind gratuities of US$250000.00, and all the required documentation will be used in the judgement of the said Attorney.    I will be contacting you to assist in this business, in cas

In [ ]:
### IF YOU NEED TO RESTART THE RUNTIME WHICH YOU DO AFTER EACH GENERATE/FINETUNE SESSION
# local files are saved
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

!nvidia-smi

TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Thu Apr  8 05:57:59 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4    

In [ ]:
# start new session and load the run name
# generate a line for testing
sess = gpt2.start_tf_sess()

gpt2.load_gpt2(sess, run_name='recon')
gpt2.generate(sess, run_name='recon',prefix = 'FROM')

Loading checkpoint checkpoint/recon/model-1000
INFO:tensorflow:Restoring parameters from checkpoint/recon/model-1000
FROM THE DOLLARS AND  REVEAL OF YOUR MAJESTY.   I AM MR. MARTINS AKUOMI , THE AUDITOR AND COMPUTING  MANAGER OF UNITED BANK OF AFRICA PLC (U.B.A). THERE IS AN  ABANDONED Bank Account in this detour's for you.  There is, however, one condition; all cheats must be  released to the original account holder before the fund can be transferred to  any bank account.  This is simple; all you need do is  to approve the transfer on our notice that you will  be responsible for paying any amount as soon as you receive the  fund.  Be forewarned; this is not the first time this kind of  business has happened to me.  This is because I have failed to clear the account over  two years ago. Prior to my death, I cleared the account with  the office of the Director of Foreign Operations and the  account has been dormant in its current condition.  This sum of US$22.5M (Twenty-two Million, Fiv

In [ ]:
# generate file text to file name
# 200 for shorter length
# lower temp for more stability in answers and high batch size for parallel processing of samples
recon_gen_file = 'gpt2_gentext_recon.txt'

gpt2.generate_to_file(sess, destination_path = recon_gen_file, model_name='124M',                      
                      length = 200,
                      temperature = 0.5, nsamples = 200, batch_size = 20)

In [ ]:
files.download(recon_gen_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!zip -r /content/file.zip /content/checkpoint

  adding: content/checkpoint/ (stored 0%)
  adding: content/checkpoint/recon/ (stored 0%)
  adding: content/checkpoint/recon/checkpoint (deflated 40%)
  adding: content/checkpoint/recon/model-1000.data-00000-of-00001 (deflated 7%)
  adding: content/checkpoint/recon/hparams.json (deflated 28%)
  adding: content/checkpoint/recon/model-1000.meta (deflated 91%)
  adding: content/checkpoint/recon/counter (stored 0%)
  adding: content/checkpoint/recon/events.out.tfevents.1617844367.76b4ea500457 (deflated 60%)
  adding: content/checkpoint/recon/encoder.json (deflated 67%)
  adding: content/checkpoint/recon/vocab.bpe (deflated 53%)
  adding: content/checkpoint/recon/model-1000.index (deflated 62%)
  adding: content/checkpoint/phish/ (stored 0%)
  adding: content/checkpoint/phish/checkpoint (deflated 40%)
  adding: content/checkpoint/phish/model-1000.data-00000-of-00001 (deflated 7%)
  adding: content/checkpoint/phish/hparams.json (deflated 28%)
  adding: content/checkpoint/phish/model-1000.met

In [ ]:
files.download("/content/file.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!zip -r /content/file_1.zip /content/samples
files.download("/content/file_1.zip")

  adding: content/samples/ (stored 0%)
  adding: content/samples/recon/ (stored 0%)
  adding: content/samples/recon/samples-601 (deflated 54%)
  adding: content/samples/recon/samples-401 (deflated 66%)
  adding: content/samples/recon/samples-201 (deflated 54%)
  adding: content/samples/recon/samples-301 (deflated 50%)
  adding: content/samples/recon/samples-901 (deflated 49%)
  adding: content/samples/recon/samples-101 (deflated 52%)
  adding: content/samples/recon/samples-701 (deflated 53%)
  adding: content/samples/recon/samples-801 (deflated 46%)
  adding: content/samples/recon/samples-501 (deflated 49%)
  adding: content/samples/phish/ (stored 0%)
  adding: content/samples/phish/samples-601 (deflated 52%)
  adding: content/samples/phish/samples-401 (deflated 54%)
  adding: content/samples/phish/samples-201 (deflated 58%)
  adding: content/samples/phish/samples-301 (deflated 48%)
  adding: content/samples/phish/samples-901 (deflated 49%)
  adding: content/samples/phish/samples-101 (

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Social Engineering Model


In [ ]:
### IF YOU NEED TO RESTART THE RUNTIME WHICH YOU DO AFTER EACH GENERATE/FINETUNE SESSION
# local files are saved
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

!nvidia-smi

gpt2.download_gpt2(model_name="124M")

In [ ]:
# write file name and define parameters
# used max steps
# refreshed the 124M model each run as to keep the attack models different
file_name = "s_train.txt"
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='124M',
              steps=1000,
              restore_from='fresh',
              run_name='se',
              print_every=10,
              sample_every=100,
              save_every=250
              )

In [ ]:
### IF YOU NEED TO RESTART THE RUNTIME WHICH YOU DO AFTER EACH GENERATE/FINETUNE SESSION
# local files are saved
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

!nvidia-smi

In [ ]:
# start new session and load the run name
# generate a line for testing
sess = gpt2.start_tf_sess()

gpt2.load_gpt2(sess, run_name='se')
gpt2.generate(sess, run_name='se',prefix = 'FROM')

In [ ]:
# generate file text to file name
# 200 for shorter length
# lower temp for more stability in answers and high batch size for parallel processing of samples
se_gen_file = 'gpt2_gentext_se.txt'

gpt2.generate_to_file(sess, destination_path = se_gen_file, model_name='124M',                      
                      length = 200,
                      temperature = 0.5, nsamples = 200, batch_size = 20)

In [ ]:
files.download(se_gen_file)

## Phishing Model


In [ ]:
### IF YOU NEED TO RESTART THE RUNTIME WHICH YOU DO AFTER EACH GENERATE/FINETUNE SESSION
# local files are saved
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

!nvidia-smi

gpt2.download_gpt2(model_name="124M")

TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Thu Apr  8 03:11:37 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4    

Fetching checkpoint: 1.05Mit [00:00, 427Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 5.53Mit/s]
Fetching hparams.json: 1.05Mit [00:00, 533Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:51, 9.64Mit/s]
Fetching model.ckpt.index: 1.05Mit [00:00, 773Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 8.76Mit/s]
Fetching vocab.bpe: 1.05Mit [00:00, 6.74Mit/s]


In [ ]:
# write file name and define parameters
# used max steps
# refreshed the 124M model each run as to keep the attack models different
file_name = "p_train.txt"
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='124M',
              steps=1000,
              restore_from='fresh',
              run_name='phish',
              print_every=10,
              sample_every=100,
              save_every=250
              )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Loading checkpoint models/124M/model.ckpt
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:04<00:00,  4.85s/it]


dataset has 1026585 tokens
Training...
[10 | 27.13] loss=3.31 avg=3.31
[20 | 49.64] loss=2.85 avg=3.08
[30 | 72.21] loss=2.98 avg=3.05
[40 | 94.97] loss=2.71 avg=2.96
[50 | 117.87] loss=2.58 avg=2.88
[60 | 140.97] loss=2.62 avg=2.84
[70 | 164.17] loss=2.80 avg=2.83
[80 | 187.49] loss=2.10 avg=2.74
[90 | 210.89] loss=2.46 avg=2.70
[100 | 234.42] loss=2.22 avg=2.65
======== SAMPLE 1 ========
 leaders to assist the victims,  and send it to your  country where the  fund-sharing will be finalized in your favour.  If the money has not been paid properly, the money will  be forfeited or forfeited if the  fund shall not be paid up  within a certain amount of time.  This  transaction would be most profitable for both of us because  I  will be working as an  auditor and the money is secured  by an  award-sharing agreement.  I  await your full  response .                     _________________________________________________________________________ DETAILS  ---------------------                   

In [ ]:
### IF YOU NEED TO RESTART THE RUNTIME WHICH YOU DO AFTER EACH GENERATE/FINETUNE SESSION
# local files are saved
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

!nvidia-smi



TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Thu Apr  8 05:54:40 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4    

In [ ]:
# start new session and load the run name
# generate a line for testing
sess = gpt2.start_tf_sess()

gpt2.load_gpt2(sess, run_name='phish')
gpt2.generate(sess, run_name='phish',prefix = 'FROM')

Loading checkpoint checkpoint/phish/model-1000
INFO:tensorflow:Restoring parameters from checkpoint/phish/model-1000
FROM OUR SON  AS HE IS THE NEXT OF KIN TO  MY LATE FATHER, HOPE THAT YOU WILL NOT TAKE MY PETITION TO  THE HEIR OF MY LATE LATE (HEIR LATE MUHAMED MOHAMED MOHAMMED)  WE ARE SEEKING FOR INFORMATION WHICH IS VERY CONFIDENTIAL BUT ALSO  GRATEFUL  DURING THE TIME OF EXECUTION AND ALSO SIT ON THE ARRANGEMENTS WHICH  YOU HAVE BEEN ASSURED TO RECEIVE THE MONEY ON YOUR  ACCOUNT. PLEASE, YOU HAVE BEEN ADVICED TO KEEP "TOP SECRET" AS I AM  STILL IN SERVICE AND INTEND TO RETIRE FROM SERVICE AFTER WE CONCLUDE  THIS DEAL WITH YOU. I WILL BE MONITORING THE WHOLE SITUATION HERE IN THIS BANK UNTIL YOU CONFIRM  THE MONEY IN YOUR ACCOUNT AND ASK ME TO COME DOWN TO YOUR COUNTRY FOR  SUBSEQUENT SHARING OF THE FUND ACCORDING TO PERCENTAGES PREVIOUSLY  ANDROID DIVIDUAL)AND FOREIGN.  PLEASE, YOU HAVE BEEN ADVICED TO KEEP "TOP SECRET" AS I AM STILL IN SERVICE AND INTEND TO  RETIRE FROM SERVICE 

In [ ]:
# start new session and load the run name
# generate a line for testing
# generate file text to file name
# 200 for shorter length
# lower temp for more stability in answers and high batch size for parallel processing of samples
phish_gen_file = 'gpt2_gentext_phish.txt'

gpt2.generate_to_file(sess, destination_path = phish_gen_file, model_name='124M',                      
                      length = 200,
                      temperature = 0.5, nsamples = 200, batch_size = 20)

In [ ]:
files.download(phish_gen_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Malware Model


In [ ]:
### IF YOU NEED TO RESTART THE RUNTIME WHICH YOU DO AFTER EACH GENERATE/FINETUNE SESSION
# local files are saved
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

!nvidia-smi

gpt2.download_gpt2(model_name="124M")

In [ ]:
# malware model finetune
file_name = "mal_train.txt"
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='124M',
              steps=1000,
              restore_from='fresh',
              run_name='mal',
              print_every=10,
              sample_every=100,
              save_every=250
              )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Loading checkpoint models/124M/model.ckpt
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:00<00:00,  2.51it/s]


dataset has 56625 tokens
Training...
[10 | 27.48] loss=2.69 avg=2.69
[20 | 49.14] loss=2.78 avg=2.74
[30 | 71.04] loss=1.79 avg=2.42
[40 | 93.26] loss=1.94 avg=2.30
[50 | 115.80] loss=1.08 avg=2.05
[60 | 138.46] loss=1.58 avg=1.97
[70 | 161.19] loss=1.27 avg=1.87
[80 | 184.12] loss=1.36 avg=1.80
[90 | 207.21] loss=0.72 avg=1.68
[100 | 230.43] loss=0.42 avg=1.55
======== SAMPLE 1 ========
 hundred percent of both the total sum and any interest which might be accrued on it at maturity before the maturity date. This deal is sometimes termed the Chase  Direct offer.  The money is already on its way to you in a checabdn form, from which it is waiting  for our Diplomatic Security and Finance Representative to reach here in Dubai.  In case you are not familiar with him or his  contact email, I recommend watching this video:  http://www.cnn.com/2010/WORLD/02/02/uk/politics/the-second-in-second-in-second-episode-1.  http://www.cnn.com/2002/WORLD/2002/WORLD/europe/  So get down to business now, 

In [ ]:
### IF YOU NEED TO RESTART THE RUNTIME WHICH YOU DO AFTER EACH GENERATE/FINETUNE SESSION
# local files are saved
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

!nvidia-smi

TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Thu Apr  8 05:47:07 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4    

In [ ]:
# start new session and load the run name
# generate a line for testing
sess = gpt2.start_tf_sess()

gpt2.load_gpt2(sess, run_name='mal')
gpt2.generate(sess, run_name='mal',prefix = 'FROM')

Loading checkpoint checkpoint/mal/model-1000
INFO:tensorflow:Restoring parameters from checkpoint/mal/model-1000
FROM THE RELEASE:From the Financial Institution:  My name is Mrs.Monica Martins, the wife of late Mr.Zulu Martins, of Liberia.We are in our possession vital approved papers to invoice Mr.Zulu Martins as the next of kin to Lassalle Dembo, an Iraqi refugee ,after which time the couple will not be related. After this Dembo died, and new laws which limited the number of refugees we could accept was introduced, lassalle decided to ban all biological materials and financial records of all Liberians within 14 working days beginning with his last known addresses. This has made my husband the next of kin in lawless Africa, and he has been since de facto the head of state for the past four decades now.  I have in mind that we keep this money for as personal protection since I cannot lodge the required documents with the relevant immigration department.  However, upon due consultation 

In [ ]:
# generate file text to file name
# 200 for shorter length
# lower temp for more stability in answers and high batch size for parallel processing of samples
mal_gen_file = 'gpt2_gentext_mal.txt'

gpt2.generate_to_file(sess, destination_path = mal_gen_file, model_name='124M',                      
                      length = 200,
                      temperature = 0.5, nsamples = 200, batch_size = 20)

In [ ]:
files.download(mal_gen_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Reply Model

In [ ]:
### IF YOU NEED TO RESTART THE RUNTIME WHICH YOU DO AFTER EACH GENERATE/FINETUNE SESSION
# local files are saved
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

!nvidia-smi

Thu Apr  8 18:07:14 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P8    31W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# reply model finetune
# using same method as the original generator models
file_name = "reply_train.txt"
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='124M',
              steps=1000,
              restore_from='fresh',
              run_name='reply',
              print_every=10,
              sample_every=100,
              save_every=250
              )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Loading checkpoint models/124M/model.ckpt
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:07<00:00,  7.94s/it]


dataset has 1329923 tokens
Training...
[10 | 51.52] loss=2.88 avg=2.88
[20 | 95.88] loss=3.02 avg=2.95
[30 | 140.16] loss=2.78 avg=2.89
[40 | 184.44] loss=2.72 avg=2.85
[50 | 228.65] loss=2.81 avg=2.84
[60 | 272.91] loss=2.93 avg=2.86
[70 | 317.19] loss=2.57 avg=2.81
[80 | 361.40] loss=2.80 avg=2.81
[90 | 405.62] loss=2.79 avg=2.81
[100 | 449.87] loss=2.47 avg=2.77
======== SAMPLE 1 ========
 we  will have  a strong interest to  take a little  hard road.  Please take a picture  of them together -  let me know when  they arrive to see if they have any fun!!!\nP.S. - If you were unable to make this for the  conference  on Wednesday and Thursday then make sure that we  are in the conference room on Wednesday and Thursday\nWednesday-Saturday, 5-6pm.  We will get back to you quickly.\nJeff\n\t\t \nJeff (\nThe e-mail in this message is confidential.  It is the only e-mail that is \ntransmittable to others.', 'It's about time for  them to be in.  \n\t\t \n\t \n\t \n ', "Well it'll be fine if 

In [ ]:
### IF YOU NEED TO RESTART THE RUNTIME WHICH YOU DO AFTER EACH GENERATE/FINETUNE SESSION
# local files are saved
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

!nvidia-smi

Thu Apr  8 18:07:59 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P8    31W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# test out the replies and load sesion
sess = gpt2.start_tf_sess()

gpt2.load_gpt2(sess, run_name='reply')
gpt2.generate(sess, run_name='reply',prefix = 'FROM')

Loading checkpoint checkpoint/reply/model-1000
INFO:tensorflow:Restoring parameters from checkpoint/reply/model-1000
FROMMING IN THE DEPARTMENT OF HEALTH and\nHEALTH SERVICES,\nHELP ME WITH AUSTIN, PUCIGAN STATE  FEMALI\nHELP ME WITH     AUSTRALIA\nHAVE FUNNY I\'m sorry I didn\'t get back to you yesterday, but I wanted to \nhave a big heart meeting with you at your house this evening to discuss our \nhearings regarding the PUC's activities over the past year.  I\'m going to \nmake sure that everything is moving forward and that things are going well.\nThere are a few things that need to happen.  I\'m going to need the following \ninformation;\n1)  We have been working with some of the biggest names in the energy industry on a new \nsystem for valuation and payment incentive.  These guys are really great.  I just want to \nmake sure that everyone is being paid the same amount.  We are going to be very \naggressive, but I\'m not going to put anyone out of work.\n2)  We have been talking 

In [ ]:
# generate to file with THREE REPLIES per 800 emails
reply_gen_file = 'gpt2_gentext_reply.txt'

gpt2.generate_to_file(sess, destination_path = reply_gen_file, model_name='124M',                      
                      length = 200,
                      temperature = 0.5, nsamples = 2400, batch_size = 20)

In [ ]:
files.download(reply_gen_file)

# Force-kill VM

If the notebook has errors (e.g. GPU Sync Fail), force-kill the Colaboratory virtual machine and restart it with the command below:

In [ ]:
!kill -9 -1

# LICENSE

MIT License

Copyright (c) 2019 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

##  Train a GPT-2 Text-Generating Model w/ GPU For Free 

by [Max Woolf](http://minimaxir.com)

*Last updated: February 14th, 2021*

Retrain an advanced text generating neural network on any text dataset **for free on a GPU using Collaboratory** using `gpt-2-simple`!

For more about `gpt-2-simple`, you can visit [this GitHub repository](https://github.com/minimaxir/gpt-2-simple). You can also read my [blog post](https://minimaxir.com/2019/09/howto-gpt2/) for more information how to use this notebook!
